In [15]:
# 네이버 영화 리뷰 모으기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy as np
import pandas as pd  
import os
from datetime import datetime
import xlwt

#Step 2. 사용자에게 검색어 키워드를 입력 받고 저장할 폴더와 파일명을 설정합니다.
print("=" *80)
print(" 네이버 영화의 리뷰 수집하기")
print("=" *80)

query_txt = input('1.크롤링 할 영화 제목을 입력하세요:')
cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://movie.naver.com")
time.sleep(2)

element = driver.find_element_by_class_name("ipt_tx_srch")
element.send_keys(query_txt)
driver.find_element_by_class_name("btn_srch").click()
time.sleep(2)
driver.find_element_by_xpath("""//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a""").click()
driver.find_element_by_xpath("""//*[@id="movieEndTabMenu"]/li[5]/a""").click()

element = driver.find_element_by_class_name("ipt_tx_srch")
element.send_keys(query_txt)
driver.find_element_by_class_name("btn_srch")

#Step 4. 현재 총 리뷰 건수를 확인하여 사용자의 요청건수와 비교 후 동기화합니다

# iframe 이동
driver.switch_to_frame('pointAfterListIframe')

# 전체 소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('strong','total').find_all('em')

result2= result[1]

result3 = result2.get_text()

print("=" *80)

result4 = result3.replace("\t",'')

result5 = result4.replace(',','')

result6 = result5.replace('\n','')

search_cnt = int(result6)

if cnt > search_cnt:
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)
print("실체 출력될 최종 페이지수" , page_cnt)

# 크롤링한 글 수 카운트
count = 0

# 칼럼 리스트 준비
score = []
text = []
user = []
date = []
good = []
bad = []

while(True):

    # 리뷰 리스트 가져오기
    content_list =  soup.find('div',class_='ifr_area basic_ifr').find('div', class_ = 'score_result').find('ul').find_all('li')

    for li in content_list:

        count += 1

        # 각 요소 가져오기
        tmp_score = li.find('div', class_='star_score').find('em').text
        tmp_text = li.find('div', class_='score_reple').find('p').text
        tmp_user = li.find('div', class_='score_reple').find('dl').find('span').text
        tmp_date = li.find('div', class_='score_reple').find_all('em')[1].text
        tmp_good = li.find('div', class_='btn_area').find_all('strong')[0].text
        tmp_bad = li.find('div', class_='btn_area').find_all('strong')[1].text

        # 칼럼 리스트에 추가
        score.append(tmp_score)
        text.append(tmp_text)
        user.append(tmp_user)
        date.append(tmp_date)
        good.append(tmp_good)
        bad.append(tmp_bad)

        # 확인용 프린트
        print("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다===================================="%(cnt, count))
        print('1) 별점:', tmp_score)
        print('2) 리뷰내용:', tmp_text)
        print('3) 작성자:', tmp_user)
        print('4) 작성일자:', tmp_date)
        print('5) 공감:', tmp_good)
        print('6) 비공감:',tmp_bad)
        print('\n')

        # txt파일에 저장
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다===================================="%(cnt, count) + '\n')
        f.write('1) 별점: ' + tmp_score + '\n')
        f.write('2) 리뷰내용: ' + tmp_text + '\n')
        f.write('3) 작성자: ' + tmp_user + '\n')
        f.write('4) 작성일자: ' + tmp_date + '\n')
        f.write('5) 공감: ' + tmp_good + '\n')
        f.write('6) 비공감: ' + tmp_bad + '\n')
        f.write('\n')

        # 만약 현재 글 수가 입력건수에 도달하면 루프 종료
        if(count == cnt):
            break
    
    if(count == cnt):
        break
    
    # 아직 입력건수에 도달하지 않았다면 다음 페이지를 열고 루프 계속
    else:
        driver.find_element_by_class_name('pg_next').click()
        time.sleep(1)
                
        driver.switch_to_default_content()
        driver.switch_to_frame('pointAfterListIframe')

        full_html = driver.page_source
        soup = BeautifulSoup(full_html, 'html.parser')
        
# 데이터프레임 생성, 각 칼럼 리스트 넣기   
df = pd.DataFrame()
df['별점'] = score
df['리뷰내용'] = text
df['작성자'] = user
df['작성일자'] = date
df['공감'] = good
df['비공감'] = bad

# 엑셀과 csv로 저장
df.to_excel(fx_name,encoding="utf-8-sig",index=True)
df.to_csv(fc_name,encoding="utf-8-sig",index=True)
                
                
print('\n모든 과정이 완료되었습니다.')

 네이버 영화의 리뷰 수집하기
1.크롤링 할 영화 제목을 입력하세요:엔드게임
2.크롤링 할 건수는 몇건입니까?: 30
3.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):C:\Temp\


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:61: DeprecationWarning: use driver.switch_to.frame instead


전체 검색 결과 건수 : 67454 건
실제 최종 출력 건수 30
실체 출력될 최종 페이지수 3
총 30 건 중 1 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 마블 3000만큼 사랑합니다  
3) 작성자: 전은호(eunh****)
4) 작성일자: 2019.04.24 10:31
5) 공감: 22009
6) 비공감: 594


총 30 건 중 2 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: Avengers Assemble!  
3) 작성자: 송(hiuh****)
4) 작성일자: 2019.04.24 09:35
5) 공감: 14479
6) 비공감: 402


총 30 건 중 3 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: I'm Ironman.  
3) 작성자: ubola(ubol****)
4) 작성일자: 2019.04.24 10:32
5) 공감: 14104
6) 비공감: 412


총 30 건 중 4 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 관람객아이언맨으로 시작해서 아이언맨으로 끝난다..  
3) 작성자: 해피사과(qord****)
4) 작성일자: 2019.04.24 11:05
5) 공감: 12064
6) 비공감: 501


총 30 건 중 5 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 아스가디언즈 오브 갤럭시 벌써부터 기대됨 ㅋㅋㅋㅋ  
3) 작성자: 마즈(choi****)
4) 작성일자: 2019.04.24 10:47
5) 공감: 10962
6) 비공감: 391


총 30 건 중 6 번째 리뷰 데이터를 수집합니다===============

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:159: DeprecationWarning: use driver.switch_to.default_content instead
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:160: DeprecationWarning: use driver.switch_to.frame instead


총 30 건 중 11 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 영화 늦게 볼 수밖에 없는사람들을 생각해서 스포 자제해주세요ㅠㅠㅠ  
3) 작성자: 한승원(brya****)
4) 작성일자: 2019.04.24 09:07
5) 공감: 7052
6) 비공감: 590


총 30 건 중 12 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: [ 토니에게 따뜻한 마음이 있었다는 증거 ]  
3) 작성자: 김나현(gree****)
4) 작성일자: 2019.04.24 13:16
5) 공감: 6563
6) 비공감: 160


총 30 건 중 13 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 토르 등장씬보다 멋있는 어벤져스 집결씬  
3) 작성자: jjw0****
4) 작성일자: 2019.04.24 10:40
5) 공감: 6167
6) 비공감: 163


총 30 건 중 14 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 내 어린시절 11년을 즐겁게 해준 어벤져스 원년 멤버들... 마블 영화 한편 한편이 저의 초중고 시절, 울고 웃었던 추억이 되었어요. 아 눈물날거같애. 그동안 즐거웠고 감사했습니다.  
3) 작성자: 도망노비(rlad****)
4) 작성일자: 2019.04.24 09:41
5) 공감: 5354
6) 비공감: 189


총 30 건 중 15 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 관람객고맙다.나의 20대를 채워줘서  
3) 작성자: pkt(anie****)
4) 작성일자: 2019.04.24 10:51
5) 공감: 4913
6) 비공감: 152
